In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Hello! Welcome! I'm glad you're here. How can I assist you today?


In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [9]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me put on my thinking cap for this one... It’s 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [10]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [12]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [13]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website showcases his work as a programmer and innovator in the field of Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on applying AI for talent discovery and management. Previously, he founded the startup untapt, which was acquired in 2021.\n\nThe website includes various educational offerings related to LLMs, such as courses and workshops. Notable upcoming events and resources include:\n- **Connecting my courses – become an LLM expert and leader** (May 28, 2025)\n- **2025 AI Executive Briefing** (May 18, 2025)\n- **The Complete Agentic AI Engineering Course** (April 21, 2025)\n- **LLM Workshop – Hands-on with Agents – resources** (January 23, 2025)\n\nAdditionally, the site expresses his interests in coding, DJing, electronic music production, and engaging with the tech community through platforms like Hacker News. "

In [14]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

**About Edward Donner**  
Edward Donner is a co-founder and CTO of Nebula.io, a company focused on using AI to help individuals discover their potential and effectively manage talent. He previously founded the AI startup untapt, which was acquired in 2021. In addition to coding and LLM experimentation, he enjoys DJing and electronic music production.

**Key Features**  
- **Connect Four**: A platform where LLMs compete in diplomacy and cunning challenges.
- Various posts and announcements regarding courses and workshops related to LLMs (Large Language Models).

### Recent Announcements
- **May 28, 2025**: Launching a course aimed at developing expertise and leadership in LLM.
- **May 18, 2025**: Details about the 2025 AI Executive Briefing.
- **April 21, 2025**: Introduction of "The Complete Agentic AI Engineering Course."
- **January 23, 2025**: A workshop providing hands-on resources for working with agents in LLMs. 

**Connect**  
Visitors are encouraged to connect via email and social media platforms like LinkedIn and Twitter.

In [16]:
display_summary("https://cnn.com")

# CNN Website Summary

The CNN website serves as a comprehensive platform for the latest news and updates across various topics, including US and world news, politics, business, health, entertainment, style, sports, science, and climate. The homepage features top stories, live updates, and trending articles.

## Recent News Highlights:
- **Israel's Gaza Offensive**: The Israeli military has intensified its operations in Gaza amidst a spiraling humanitarian crisis.
- **US Citizen Executed in Syria**: An American among a family of eight Druze men was executed amid violent sectarian clashes in Syria.
- **Trump Administration**: Discussions about the Department of Justice reaching out to Ghislaine Maxwell for testimony related to Jeffrey Epstein's case.
- **Venus Williams’ Return**: At the age of 45, tennis star Venus Williams returned to competitive play successfully.
- **UNESCO Withdrawal**: The US plans to withdraw from the UN scientific and cultural agency again, as announced by the White House.

### Additional Noteworthy Stories:
- The potential ramifications of climate change on global food prices.
- A very high level of toxic chemicals found in eco-friendly menstrual products.
- Insights into Gen Z's concept of 'quiet relationships' shaped by social media.
- A significant rise in interest around geothermal energy as a renewable resource.

For more live news and updates, users can access CNN's various sections or subscribe to newsletters tailored to their interests.

In [17]:
display_summary("https://www.forbes.com/profile/larry-ellison/?list=rtb/")

# Summary of Larry Ellison Website

The website provides an overview of Larry Ellison, the co-founder and Chief Technology Officer of Oracle Corporation. It details his professional achievements, wealth status, and personal background.

## Key Highlights:
- **Current Wealth**: Larry Ellison has a net worth of **$283.8 billion**, making him the **2nd richest person** in the world as of July 22, 2025.
- **Professional Background**: Ellison founded Oracle and held the CEO position for 37 years before stepping down in 2014. He remains a significant stakeholder in the company, owning approximately 40% of its shares.
- **Investments and Ventures**: He plans to invest **$5 billion** in cloud infrastructure in the UK to address the growing demand for Oracle services. 
- **Residence**: Ellison lives on the Hawaiian island of Lanai, which he purchased in 2012, and has a notable interest in luxury real estate.

## Recent News:
- On **June 13, 2025**, a significant rise in Oracle's stock led to a $40 billion increase in Ellison's wealth, surpassing notable figures like Jeff Bezos and Mark Zuckerberg.
- Other news articles highlight Ellison's fluctuating wealth as influenced by market trends and stock performance of Oracle and associated companies.

## Personal Insights:
- **Education**: Ellison is a college dropout, having attended the University of Chicago and the University of Illinois, Urbana-Champaign.
- **Family**: He has four children, with notable achievements in media and entertainment.

Overall, the website presents a comprehensive profile of Larry Ellison as a key figure in technology and business, focusing on his accomplishments, investments, and significant current events related to him and Oracle.

In [19]:
# Step 1: Create your prompts

system_prompt = "You are an assistant whose task is to read an email (official puprpose) and suggest a subject line for the email "
user_prompt = """
Provide a sample line for below mail
To:
Embassy of Germany
Consular Section
17, Nyaya Marg, Chanakyapuri
New Delhi - 110021

Date: June 16, 2025

Dear Sirs,

This letter is to present Ms. Ananya Rao, Business Development Manager at InnovaSoft Solutions Pvt. Ltd.

The purpose of Ms. Ananya Rao’s visit to Germany is to represent InnovaSoft Solutions Pvt. Ltd. on a business development and client onboarding trip with our German partner firm.

The relevant information pertaining to this visit is as follows:

Name of applicant: Ananya Rao

Nationality: Indian

Passport #: M1234567

Date of issue of passport: January 10, 2022

Expiry date: January 09, 2032

Place of birth: Bengaluru, India

Date of birth: April 15, 1991

Occupation: Business Development Manager

Name of inviting company: TechHub GmbH, Berlin

Period of stay: July 5, 2025 – July 19, 2025

We kindly ask you to grant Ms. Ananya Rao the appropriate short-term Schengen business visa for the above-mentioned purpose. Please be advised that InnovaSoft Solutions Pvt. Ltd. will accept full financial responsibility for Ms. Ananya Rao during this visit.

Thank you.

Yours truly,
Rakesh Mehta
Director – International Affairs
InnovaSoft Solutions Pvt. Ltd.

"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)

Subject: Request for Short-Term Schengen Business Visa for Ms. Ananya Rao
